In [6]:
import os 
from pathlib import Path
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

In [4]:
df = pd.read_csv('./data/train/train.csv')
df

,ImageId,NameOfPokemon
0,296.jpg,Pikachu
1,297.jpg,Pikachu
2,298.jpg,Pikachu
3,299.jpg,Pikachu
4,300.jpg,Pikachu
...,...,...
299,616.jpg,Charmander
300,617.jpg,Charmander
301,618.jpg,Charmander
302,619.jpg,Charmander


In [5]:
labels_dict={"Pikachu":0,"Bulbasaur":1,"Meowth":2 }
label2pokemon = {0:"Pikachu",1:"Bulbasaur",2:"Meowth"}


# Initialize lists to store images and labels
X = []
Y = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Image name
    image_name = row['Image_Name']
    
    # Pokemon name
    pokemon_name = row['Pokemon_Name']
    
    # Load image
    image_path = os.path.join('./data/train/', image_name)
    image = Image.open(image_path)
    
    # Convert image to numpy array or any other desired format
    # image_array = np.array(image)
    
    # Append image and label to X and Y lists
    X.append(image)
    Y.append(pokemon_name)

# Optionally, you can convert X and Y to numpy arrays if needed
# X = np.array(X)
# Y = np.array(Y)

# Now, X contains the images and Y contains their corresponding labels
